In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [12]:
# Libraries
# ======================================================================================
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OrdinalEncoder

In [37]:
# Data
# ======================================================================================
y = pd.Series(np.random.normal(size=50))
exog_1 = pd.Series([1]*25 + [2]*25, name='exog_1', dtype=int)
exog_2 = pd.Series(['a']*25 + [1]*25, name='exog_2', dtype=str)
exog   = pd.concat((exog_1, exog_2), axis=1)
exog.head()

,exog_1,exog_2
0,1,a
1,1,a
2,1,a
3,1,a
4,1,a


In [38]:
forecaster = ForecasterAutoreg(regressor=LGBMRegressor(random_state=12345), lags=5)
forecaster = ForecasterAutoreg(regressor=LinearRegression(), lags=5)
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog
)
print(X_train.dtypes)
X_train.head(3)

lag_1     float64
lag_2     float64
lag_3     float64
lag_4     float64
lag_5     float64
exog_1      int64
exog_2     object
dtype: object


,lag_1,lag_2,lag_3,lag_4,lag_5,exog_1,exog_2
5,1.473746,-1.615842,-1.555698,1.243674,-1.282704,1,a
6,0.138526,1.473746,-1.615842,-1.555698,1.243674,1,a
7,-0.209535,0.138526,1.473746,-1.615842,-1.555698,1,a


In [39]:
forecaster.fit(y=y, exog=exog)
exog_predict = exog.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

ValueError: could not convert string to float: 'a'

In [23]:
categorical_cols = exog.select_dtypes(include="category").columns
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(dtype=int, handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")


exog_encoded = ordinal_encoder.fit_transform(exog)
exog_encoded[categorical_cols] = exog_encoded[categorical_cols].astype('category')

In [24]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog_encoded
)
print(X_train.dtypes)
X_train.head(3)

In [25]:
forecaster.fit(y=y, exog=exog_encoded)
exog_predict = exog_encoded.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

In [28]:
ordinal_encoder = make_column_transformer(
                        (
                            OrdinalEncoder(dtype=int, handle_unknown="use_encoded_value", unknown_value=-1),
                            make_column_selector(dtype_include=["object", "category"]),
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                    ).set_output(transform="pandas")


forecaster = ForecasterAutoreg(
    regressor = LGBMRegressor(),
    lags = 5,
    transformer_exog=OrdinalEncoder(dtype=int, handle_unknown="use_encoded_value", unknown_value=-1)
)

forecaster.fit(y=y, exog=exog_2)
exog_predict = exog_2.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

50   -0.224048
51    0.325645
52   -0.425305
Name: pred, dtype: float64